In [48]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(),os.pardir))# TODO: change when change to python file
import pandas as pd
import torch
import recipe_1m_analysis.utils as utils
import numpy as np
import math
import matplotlib.pyplot as plt
from datetime import datetime
import csv

from sklearn import metrics
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader, Sampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Data preprocessing

In [4]:
FOLDER_PATH = "D:\\Google Drive\\Catherning Folder\\THU\\Thesis\\Recipe datasets\\"
DATASET = ["scirep-cuisines-detail","Yummly28"]
FILES = ["random","cluster_centroid","full"]

file = FILES[2]
balanced = False
load = False

In [45]:
dataset = DATASET[1]

df=pd.read_pickle(os.path.join(FOLDER_PATH,dataset,file+"_data.pkl"))
df=df.set_index("id")

In [46]:
dataset = DATASET[0]

df2=pd.read_pickle(os.path.join(FOLDER_PATH,dataset,file+"_data.pkl"))
df2["id"]=[len(df)+i for i in range(len(df2))]
df2=df2.set_index("id")
df = pd.concat([df, df2])
df

# TODO: Try by only selecting recipes where ingredients are in vocab_ingr from recipe1M ?

,cuisine,ingredients,all_ingredients
id,,,
0,Italian,"[baby_bella_mushroom_sliced, arborio_rice, oli...",baby_bella_mushroom_sliced;arborio_rice;olive_...
2,Italian,"[garlic, extra-virgin_olive_oil_divided, madei...",garlic;extra-virgin_olive_oil_divided;madeira_...
3,French,"[garlic_halved, cooking_spray, peeled_red_pota...",garlic_halved;cooking_spray;peeled_red_potato_...
5,American,"[turkey_thawed, maple_syrup, bourbon, pickling...",turkey_thawed;maple_syrup;bourbon;pickling_spi...
6,American,"[beef_tenderloin__silver_skin_trimmed, worcest...",beef_tenderloin__silver_skin_trimmed;worcester...
...,...,...,...
84008,Italian,"[kiwi, olive_oil, clam, white_wine, orange, sa...",kiwi;olive_oil;clam;white_wine;orange;salmon;f...
84009,Italian,"[tomato, butter, beef, onion, red_wine, black_...",tomato;butter;beef;onion;red_wine;black_pepper...
84010,Italian,"[vegetable, wheat, egg, cheese, olive_oil]",vegetable;wheat;egg;cheese;olive_oil


In [49]:
vocab_ingrs = utils.Vocabulary()
for ingredients in df.loc[:,"ingredients"]:
    for ingr in ingredients:
        vocab_ingrs.add_word(ingr)
vocab_ingrs.add_word("<unk>")
vocab_ingrs.word2idx

{'baby_bella_mushroom_sliced': 0,
 'arborio_rice': 1,
 'olive_oil': 2,
 'butter': 3,
 'shallot_minced': 4,
 'white_wine': 5,
 'fat_free_chicken_stock_': 6,
 'salt_pepper': 7,
 'grated_parmesan_cheese': 8,
 'chopped_parsley': 9,
 'garlic': 10,
 'extra-virgin_olive_oil_divided': 11,
 'madeira_divided': 12,
 'unsalted_chicken_stock_divided': 13,
 'dried_porcini_mushroom_': 14,
 'chopped_onion': 15,
 'thinly_sliced_cremini_mushroom_': 16,
 'thinly_sliced_shiitake_mushroom_cap_': 17,
 'uncooked_arborio_rice': 18,
 'parmesan_cheese_grated_': 19,
 'kosher_salt': 20,
 'freshly_ground_black_pepper': 21,
 'chopped_fresh_sage': 22,
 'sage_': 23,
 'garlic_halved': 24,
 'cooking_spray': 25,
 'peeled_red_potato_': 26,
 'butter_melted': 27,
 'salt': 28,
 'black_pepper': 29,
 'shredded_gruyere_cheese': 30,
 'fat-free_milk': 31,
 'turkey_thawed': 32,
 'maple_syrup': 33,
 'bourbon': 34,
 'pickling_spice': 35,
 'large_carrot_scraped': 36,
 'celery_rib': 37,
 'medium_onion_peeled_halved': 38,
 'lemon': 39

In [51]:
vocab_cuisine = utils.Vocabulary()
for cuisine in df['cuisine'].value_counts().index:
    vocab_cuisine.add_word(cuisine)
vocab_cuisine.word2idx

{'American': 0,
 'Italian': 1,
 'Mexican': 2,
 'Asian': 3,
 'French': 4,
 'Indian': 5,
 'east_asian': 6,
 'Korean': 7,
 'Canada': 8,
 'western': 9,
 'Chinese': 10,
 'Mediterranean': 11,
 'Spanish_Portuguese': 12,
 'Eastern-Europe': 13,
 'Southern_SoulFood': 14,
 'Jewish': 15,
 'Japanese': 16,
 'Thai': 17,
 'Germany': 18,
 'UK-and-Ireland': 19,
 'Scandinavian': 20,
 'MiddleEastern': 21,
 'Central_SouthAmerican': 22,
 'Greek': 23,
 'English_Scottish': 24,
 'Caribbean': 25,
 'Moroccan': 26,
 'Cajun_Creole': 27,
 'African': 28,
 'Southwestern': 29,
 'South-America': 30,
 'Vietnamese': 31,
 'Irish': 32,
 'North-African': 33,
 'Portugal': 34,
 'Philippines': 35,
 'Netherlands': 36,
 'Lebanon': 37,
 'Cuban': 38,
 'Iran': 39,
 'Austria': 40,
 'Switzerland': 41,
 'Pakistan': 42,
 'Malaysia': 43,
 'asian': 44,
 'South-African': 45,
 'Turkey': 46,
 'English_Irish': 47,
 'West-African': 48,
 'Indonesia': 49,
 'East-African': 50,
 'Belgium': 51}

## Data processing

In [52]:
def ingr2idx(ingr_list):
    # If I didn't do the one-hot encoding by myself and used directly an embedding layer in the net, 
    # I would have to pad the input
    input_=[]
    for ingr in ingr_list:
        try:
            input_.append(vocab_ingrs.word2idx[ingr])
        except KeyError:
            input_.append(vocab_ingrs.word2idx["<unk>"])
    input_ = torch.LongTensor(input_)
    onehot_enc = F.one_hot(input_.to(torch.int64), INPUT_SIZE)
    output = torch.sum(onehot_enc,0)
    return output

class IngrDataset(Dataset):
    """Recipes dataset for cuisine classification. Only from ingredients for now"""

    def __init__(self, input_,labels):
        """
        Args:
            file (string): Path to the file
        """
        self.input_ = input_
        self.labels = labels

    def __len__(self):
        return len(self.input_)

    def __getitem__(self, idx):
        # XXX useful ?
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        ingr_one_hot = ingr2idx(self.input_.loc[idx,"ingredients"])
        label = vocab_cuisine.word2idx[self.labels.loc[idx,"cuisine"]]
        return ingr_one_hot, label
        

In [53]:
def make_weights_for_balanced_classes(samples, nclasses): 
    count = [0] * nclasses
    weight_per_class = [0.] * nclasses
    N = len(samples)
    
    for a,el in samples.value_counts().items():
        count[vocab_cuisine.word2idx[a]]=el

    for i in range(nclasses): # XXX to still sample the others, add log ? so goes back to N, not max(count)
        weight_per_class[i] = math.log(N/float(count[i])) # divide by max count[i] ? Or just different scale, order is same
        print(vocab_cuisine.idx2word[i], weight_per_class[i])
    weight = [0] * N
    
    for idx, val in enumerate(samples): 
        weight[idx] = weight_per_class[vocab_cuisine.word2idx[val]] 
        
    
    return torch.Tensor(weight_per_class), torch.DoubleTensor(weight)

# from https://gist.github.com/srikarplus/15d7263ae2c82e82fe194fc94321f34e

In [54]:
INPUT_SIZE = len(vocab_ingrs)
EMBED_DIM1 = 300
EMBED_DIM2 = 64
NUM_CLASSES = len(vocab_cuisine) #51 or 20
BATCH_SIZE = 100
PRINT_FREQ = 20
NB_EPOCHS = 30

Weighted random sampling, with stratified split for the train and test dataset. But loss doesn't decrease (need to see more epochs ?)

In [60]:
#TODO when switch to python file, can put num_workers & have to put if __name__ == '__main__':
X_train, X_dev, y_train, y_dev = train_test_split(df["ingredients"],df["cuisine"], test_size=0.1, random_state=42,stratify=df["cuisine"])
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train, test_size=0.1, random_state=42,stratify=y_train)

X_train = X_train.reset_index()
X_dev = X_dev.reset_index()
X_test = X_test.reset_index()
y_train = y_train.reset_index()
y_dev = y_dev.reset_index()
y_test = y_test.reset_index()

train_dataset = IngrDataset(X_train,y_train)
dev_dataset = IngrDataset(X_dev,y_dev)
test_dataset = IngrDataset(X_test,y_test)

if balanced:
    weights_classes, weights_labels = make_weights_for_balanced_classes(y_train["cuisine"], len(vocab_cuisine.word2idx)) 
    print(len(weights_labels))
    sampler = torch.utils.data.sampler.WeightedRandomSampler(weights_labels, len(weights_labels)) 
    train_loader = DataLoader(train_dataset,batch_size = BATCH_SIZE, sampler = sampler)#, pin_memory=True)
else:
    train_loader = DataLoader(train_dataset,batch_size = BATCH_SIZE)#, pin_memory=True)    

dev_loader = DataLoader(dev_dataset,batch_size = 1)
test_loader = DataLoader(test_dataset, batch_size=1)#, sampler = sampler)

# Model

In [61]:
class Net(nn.Module):
    def __init__(self, vocab_size, embedding_dim1, embedding_dim2, num_classes):
        super(Net, self).__init__()
        self.layer_1 = nn.Linear(vocab_size, embedding_dim1, bias=True)
        self.layer_2 = nn.Linear(embedding_dim1, embedding_dim1, bias=True)
#         self.layer_3 = nn.Linear(embedding_dim1, embedding_dim2, bias=True)
        self.output_layer = nn.Linear(embedding_dim1, num_classes, bias=True)

    def forward(self, x):
        out = F.relu(self.layer_1(x))
        out = F.relu(self.layer_2(out))
#         out = F.relu(self.layer_3(out))
        out = self.output_layer(out)
        return out

net = Net(INPUT_SIZE, EMBED_DIM1, EMBED_DIM2, NUM_CLASSES)

## Training

In [62]:
if balanced:
    criterion = nn.CrossEntropyLoss(weights_classes)
else:
    criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9) #change to Adam ?

In [ ]:
if load:
    net.load_state_dict(torch.load(os.path.join(FOLDER_PATH,DATASET[1],"model_logweights")))
else:
    epoch_accuracy = []

    for epoch in range(NB_EPOCHS):

        running_loss = 0.0
        correct = 0
        total = 0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data[0]
            labels = data[1]
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # print statistics
            running_loss += loss.item()
            if i % PRINT_FREQ == PRINT_FREQ-1:    # print every 2000 mini-batches
                print(f'[Epoch {epoch + 1}, Iteration {i + 1}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

            accuracy = 100 * correct / total
        epoch_accuracy.append(accuracy)

        print(f'Accuracy of the network on epoch {epoch+1}: {accuracy:.3f}')

    print('Finished Training')

[Epoch 1, Iteration 20] loss: 0.037
[Epoch 1, Iteration 40] loss: 0.028
[Epoch 1, Iteration 60] loss: 0.020
[Epoch 1, Iteration 80] loss: 0.019
[Epoch 1, Iteration 100] loss: 0.017
[Epoch 1, Iteration 120] loss: 0.018
[Epoch 1, Iteration 140] loss: 0.017
[Epoch 1, Iteration 160] loss: 0.017
[Epoch 1, Iteration 180] loss: 0.017
[Epoch 1, Iteration 200] loss: 0.017
[Epoch 1, Iteration 220] loss: 0.016
[Epoch 1, Iteration 240] loss: 0.017
[Epoch 1, Iteration 260] loss: 0.016
[Epoch 1, Iteration 280] loss: 0.016
[Epoch 1, Iteration 300] loss: 0.016
[Epoch 1, Iteration 320] loss: 0.016
[Epoch 1, Iteration 340] loss: 0.016
[Epoch 1, Iteration 360] loss: 0.016
[Epoch 1, Iteration 380] loss: 0.016
[Epoch 1, Iteration 400] loss: 0.015
[Epoch 1, Iteration 420] loss: 0.015
[Epoch 1, Iteration 440] loss: 0.015
[Epoch 1, Iteration 460] loss: 0.015
[Epoch 1, Iteration 480] loss: 0.015
[Epoch 1, Iteration 500] loss: 0.015
[Epoch 1, Iteration 520] loss: 0.015
[Epoch 1, Iteration 540] loss: 0.014
[Epoc

In [ ]:
date = datetime.now().strftime("%m-%d-%H-%M")
# RESULTS_FOLDER = os.path.join(FOLDER_PATH,dataset,f"results_{date}{balanced*'_bal'}_{file}")
RESULTS_FOLDER = os.path.join(os.getcwd(),"results",f"nn_{date}{balanced*'_bal'}_{file}")
if not os.path.exists(RESULTS_FOLDER):
    os.makedirs(RESULTS_FOLDER)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
x_points = [i for i in range(0,NB_EPOCHS)]
# epoch_accuracy2=[]
# for i in range(len(epoch_accuracy)):
#     if i%237==0:
#         epoch_accuracy2.append(epoch_accuracy[i])
p = ax.plot(x_points, epoch_accuracy, 'b')
ax.set_xlabel('epoch')
ax.set_ylabel('accuracy')
ax.set_title('Accuracy during training')
fig.show()

fig.savefig(os.path.join(RESULTS_FOLDER,'accuracy_training.png'), dpi=fig.dpi)

## Testing on dev set

In [54]:
def f2_score(y_true, y_pred, threshold=0.5):
    return fbeta_score(y_true, y_pred, 2, threshold)

def fbeta_score(y_true, y_pred, beta, threshold, eps=1e-9):
    beta2 = beta**2

    y_pred = torch.ge(y_pred.float(), threshold).float()
    y_true = y_true.float()

    true_positive = (y_pred * y_true).sum(dim=1)
    precision = true_positive.div(y_pred.sum(dim=1).add(eps))
    recall = true_positive.div(y_true.sum(dim=1).add(eps))

    return torch.mean(
        (precision*recall).
        div(precision.mul(beta2) + recall + eps).
        mul(1 + beta2))

In [59]:
def score(network, dataloader,test=False,threshold=None):
    count_unk=0
    correct = 0
    total = 0
    all_predict = []
    all_labels = []
    with torch.no_grad():
        for data in dataloader:
            inputs = data[0]

            # Removing samples where you don't know more than 2 of the ingr doesn't help the model much
            if inputs[0][vocab_ingrs.word2idx["<unk>"]]>3:
                count_unk+=1
                continue

            labels = data[1]
            outputs = network(inputs.float())
            
            if test:
                # Only taking the prediction when the model thinks it's threshold% probable that the label is x
                # Also not that good, accuracy of 42% instead of 82% on dev set 
                proba = torch.nn.functional.softmax(outputs,dim=1)
                p_max, predicted = torch.max(proba,1)
                if p_max < threshold:
                    continue
            else:
                _, predicted = torch.max(outputs.data, 1)
            
            all_predict.append(predicted)
            all_labels.append(labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy= 100 * correct / total
    print(f'Accuracy of the network on the dataset: {accuracy:.3f}% for {total} samples')

    one_hot_pred = F.one_hot(torch.LongTensor(all_predict).to(torch.int64), NUM_CLASSES)
    one_hot_lab = F.one_hot(torch.LongTensor(all_labels).to(torch.int64), NUM_CLASSES)
    fbeta_pytorch = f2_score(one_hot_pred, one_hot_lab)

    print(f'Score is {100* fbeta_pytorch:.3f}%')
    print(f'Count unknown ingr: {count_unk}')
    
    return accuracy, fbeta_pytorch

dev_accuracy, dev_fscore = score(net,dev_loader)

Accuracy of the network on the dataset: 83.593% for 2633 samples
Score is 83.593%
Count unknown ingr: 0


## On test set

In [56]:
# df_test=pd.read_pickle(os.path.join(FOLDER_PATH,DATASET[0],FILES[1]+"_data.pkl"))
# df_test = df_test[df_test["cuisine"].isin(vocab_cuisine.word2idx.keys())] #some labels not existing in train dataset
# df_test_s=df_test.sample(5000)
# df_test_s=df_test_s.reset_index()

# test_dataset= IngrDataset(pd.DataFrame(df_test_s["ingredients"]),pd.DataFrame(df_test_s["cuisine"]))

# test_loader = DataLoader(test_dataset, batch_size=1)#, sampler = sampler)

In [73]:
threshold = 0.90
test_accuracy, test_fscore = score(net,test_loader,test=True,threshold=threshold)

Accuracy of the network on the dataset: 62.946% for 3001 samples
Score is 62.946%
Count unknown ingr: 31


In [58]:
torch.save(net.state_dict(), os.path.join(RESULTS_FOLDER,"model_logweights"))

torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, os.path.join(RESULTS_FOLDER,"training_state_logweights"))

In [72]:
results_file = os.path.join(RESULTS_FOLDER"results.csv")
if os.path.isfile(results_file):
    with open(results_file,"w", newline='') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow(["date","file","balanced","epoch","train_accuracy","dev_accuracy","dev_fscore","test_accuracy","test_fscore","threshold"])
        writer.writerow([date,file,balanced,NB_EPOCHS,epoch_accuracy[-1],dev_accuracy,dev_fscore,test_accuracy,test_fscore,threshold])
else:
    with open(results_file,"a", newline='') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow([date,file,balanced,NB_EPOCHS,epoch_accuracy[-1],dev_accuracy,dev_fscore,test_accuracy,test_fscore,threshold])